In [2]:
import yaml
from yaml.loader import SafeLoader
from datetime import datetime
import time
import pandas as pd
file='Milestone1B.yaml'

with open(file) as f:
    config = yaml.load(f, Loader=SafeLoader)
print(config)

{'M1B_Workflow': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskA': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskA_Input', 'ExecutionTime': '1'}}, 'FlowA': {'Type': 'Flow', 'Execution': 'Concurrent', 'Activities': {'TaskB': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '14'}}, 'TaskC': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '5'}}, 'TaskD': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '5'}}, 'FlowB': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskE': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskE_Input', 'ExecutionTime': '5'}}, 'TaskF': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskF_Input', 'ExecutionTime': '1'}}}}, 'FlowC': {'Type': 'Flow', 'Execution': 'Sequential', 'Activ

In [3]:
def TimeFunction(FunctionInput,ExecutionTime):
    print(1)
    time.sleep(int(ExecutionTime))
    print(int(ExecutionTime))
    return
    
def DataLoad(Filename):
    print(2)
    DataTable=pd.read_csv(Filename)
    NoOfDefects=DataTable.shape[0]-1
    
    return DataTable, NoOfDefects


def callfunction(name,inputs):
    if name=='TimeFunction':
        TimeFunction(inputs[0],inputs[1])
    elif name=='DataLoad':
        DataLoad(inputs[0])

In [4]:
def runTask(task,main):
    print(task)
    name=''
    for i in main:
        name+=str(i)+'.'
    
    f.write(str(datetime.now())+';'+name[:-1]+' Entry\n')
    if task['Type']=='Task':
        fun=''
        parm=''
        inputs=[]

        for i in task['Inputs'].values():
            inputs.append(i)
            fun+=str(i)+','
        if(task['Function']=='TimeFunction'):
            parm=str(inputs[-1])+','
        else:
            parm=fun
                
        
        f.write(str(datetime.now())+';'+name[:-1]+' Executing ' +str(task['Function'])+' ('+parm[:-1]+')\n')
        callfunction(task['Function'],inputs)
        
    else:
        Activits=task['Activities']
        for i in Activits:
            par=main
            par.append(i)
            runTask(Activits[i],par)
    f.write(str(datetime.now())+';'+name[:-1]+' Exit\n')

In [6]:
task_list=list(config.keys())[0]
f = open(file[:-5]+"_Log.txt", "w")
f.write(str(datetime.now())+';'+task_list+' Entry\n')
baseActivities=config[task_list]['Activities']
for i in baseActivities:
    runTask(baseActivities[i],[task_list,i])

f.write(str(datetime.now())+';'+task_list+' Exit')
f.close()

{'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskA_Input', 'ExecutionTime': '1'}}
1
1
{'Type': 'Flow', 'Execution': 'Concurrent', 'Activities': {'TaskB': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '14'}}, 'TaskC': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '5'}}, 'TaskD': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '5'}}, 'FlowB': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskE': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskE_Input', 'ExecutionTime': '5'}}, 'TaskF': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskF_Input', 'ExecutionTime': '1'}}}}, 'FlowC': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskG': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 